# Unify rasters to a base raster

In [1]:
import rasterio
import os
from importlib.resources import files
from rasterio.enums import Resampling
from pathlib import Path
from tqdm import tqdm

from beak.utilities.io import save_raster, create_file_folder_list, create_file_list, check_path
from beak.utilities.raster_processing import unify_raster_grids

In [2]:
BASE_PATH = files("beak.data")
BASE_NAME = "MCCAFFERTY23"
BASE_SPATIAL = "EPSG_32615_RES_50_0"

input_folder = BASE_PATH / BASE_NAME / "RAW"
output_folder = BASE_PATH / BASE_NAME / "PROCESSED" / BASE_SPATIAL / "GEOPHYSICS"
base_raster = BASE_PATH / "BASE_RASTERS" / str(BASE_SPATIAL + ".tif")

print(f"Input folder: {input_folder}")
print(f"Output folder: {output_folder}")
print(f"Base raster: {base_raster}")


Input folder: s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\RAW
Output folder: s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\PROCESSED\EPSG_32615_RES_50_0\GEOPHYSICS
Base raster: s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\BASE_RASTERS\EPSG_32615_RES_50_0.tif


In [4]:
# Create file and folder list
folders, _ = create_file_folder_list(input_folder)
folders.insert(0, input_folder)

file_list = []
for folder in folders:
  folder_files = create_file_list(folder, recursive=False)
  file_list.extend(folder_files)
  
print(f"Found {len(file_list)} files in {len(folders)} folders:")

for file in file_list:
  print(file)

Found 16 files in 10 folders:
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\RAW\Geophysics\Gravity.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\RAW\Geophysics\Gravity_HGM.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\RAW\Geophysics\Gravity_Up30km.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\RAW\Geophysics\Gravity_Up30km_HGM.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\RAW\Geophysics\LAB.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\RAW\Geophysics\LAB_HGM_cog.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\MCCAFFERTY23\RAW\Geophysics\LAB_Worms_Proximity.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitun

In [5]:
dry_run = False
if dry_run:
  print("Dry run, no files will be written.\n")
  
for file in tqdm(file_list, total=len(file_list)):
  out_path = output_folder / file.name
  check_path(Path(os.path.dirname(out_path)))
  
  raster = rasterio.open(file)
  unified_raster, meta = unify_raster_grids(base_raster, [file], resampling_method=Resampling.bilinear, same_extent=True, same_shape=True)[0]

  if not dry_run:
    save_raster(out_path, array=unified_raster, dtype="float32", metadata=meta)


100%|██████████| 16/16 [02:48<00:00, 10.55s/it]
